In [2]:
import torch
from tqdm.auto import tqdm
from transformer_lens import HookedTransformer
from jaxtyping import Float, Int, Bool
from torch import Tensor
from tqdm.auto import tqdm
import plotly.io as pio
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import numpy as np
import plotly.express as px 
from collections import defaultdict
import matplotlib.pyplot as plt
import re
from IPython.display import display, HTML
from datasets import load_dataset
from collections import Counter
import pickle
import os
import haystack_utils
from transformer_lens import utils
from fancy_einsum import einsum
import einops
import json
import ipywidgets as widgets
from IPython.display import display
from datasets import load_dataset
import random
import math
import random
import neel.utils as nutils
from neel_plotly import *

SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

pio.renderers.default = "notebook_connected+notebook"
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

%reload_ext autoreload
%autoreload 2

In [3]:
model: HookedTransformer = HookedTransformer.from_pretrained("gpt2-large")

data = load_dataset("stas/openwebtext-10k", split="train")
strings = [i for i in data["text"] if len(i)>2000]
len(strings)

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-large into HookedTransformer


Repo card metadata block was not found. Setting CardData to empty.


/opt/conda/lib/python3.10/site-packages/datasets/download/download_manager.py:527: FutureWarning:

'num_proc' was deprecated in version 2.6.2 and will be removed in 3.0.0. Pass `DownloadConfig(num_proc=<num_proc>)` to the initializer instead.



Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

7083

In [4]:
num_prompts = 64
n_ctx = 256
tokens = model.to_tokens(strings[num_prompts:2*num_prompts])[:, :n_ctx]
tokens.shape

logits = model(tokens)
A = model.to_single_token(" a")
AN = model.to_single_token(" an")
logit_diff = logits[:, :, AN] - logits[:, :, A]
# -5 where token is "an", diff otherwise
logit_diff = torch.where(tokens==AN, torch.zeros_like(logit_diff)-5, logit_diff)

In [5]:
token_df = pd.DataFrame({"logit_diff": logit_diff.flatten().cpu().numpy(), "batch": [b for b in range(num_prompts) for p in range(n_ctx)], "pos": [p for b in range(num_prompts) for p in range(n_ctx)]})
nutils.show_df(token_df.sort_values("logit_diff", ascending=False).head(100))

,logit_diff,batch,pos
436,7.321955,1,180
12375,3.947590,48,87
12373,3.835551,48,85
12391,3.288544,48,103
12439,2.976463,48,151
12389,2.804193,48,101
12437,2.747065,48,149
6655,2.399273,25,255
16157,2.126307,63,29
12666,1.691870,49,122


In [6]:
temp_df = token_df.sort_values("logit_diff", ascending=False).head(20)
import circuitsvis as cv
for row in temp_df.iterrows():
    b = int(row[1]["batch"])
    p = int(row[1]["pos"])
    #print(b, p)
    str_toks = model.to_str_tokens(tokens[b, :p+3])
    vals = logit_diff[b, :p+3]
    haystack_utils.print_strings_as_html(str_toks, vals.tolist(), max_value=5)
    #nutils.create_html(str_toks, vals)
    #display(cv.tokens.colored_tokens(str_toks, vals))


In [7]:
utils.test_prompt("Ross Perot, who ran as", "an", model)
utils.test_prompt("Van der Bellen, who ran as", "an", model)
utils.test_prompt("Senator Angus King, who ran as", "an", model)
utils.test_prompt("Spot gold was up 0.7 percent at $1,340.90", "an", model)
utils.test_prompt("An eye for", "an", model)
utils.test_prompt("Every action has", "an", model)
utils.test_prompt("I'll be there in under half", "an", model)
utils.test_prompt("I climbed up the pear tree and picked a pear. I climbed up the apple tree and picked", "an", model)

Tokenized prompt: ['<|endoftext|>', 'Ross', ' Per', 'ot', ',', ' who', ' ran', ' as']
Tokenized answer: [' an']


Performance on answer token:
Rank: 0        Logit: 19.55 Prob: 40.33% Token: | an|

Top 0th token. Logit: 19.55 Prob: 40.33% Token: | an|
Top 1th token. Logit: 19.40 Prob: 34.84% Token: | a|
Top 2th token. Logit: 18.92 Prob: 21.50% Token: | the|
Top 3th token. Logit: 14.48 Prob:  0.25% Token: | "|
Top 4th token. Logit: 14.37 Prob:  0.23% Token: | independent|
Top 5th token. Logit: 14.29 Prob:  0.21% Token: | one|
Top 6th token. Logit: 14.24 Prob:  0.20% Token: | Republican|
Top 7th token. Logit: 13.91 Prob:  0.14% Token: | part|
Top 8th token. Logit: 13.74 Prob:  0.12% Token: | president|
Top 9th token. Logit: 13.63 Prob:  0.11% Token: | governor|


Ranks of the answer tokens: [(' an', 0)]

Tokenized prompt: ['<|endoftext|>', 'Van', ' der', ' Bell', 'en', ',', ' who', ' ran', ' as']
Tokenized answer: [' an']


...

Ranks of the answer tokens: [(' an', 0)]

Tokenized prompt: ['<|endoftext|>', 'I', ' climbed', ' up', ' the', ' pear', ' tree', ' and', ' picked', ' a', ' pear', '.', ' I', ' climbed', ' up', ' the', ' apple', ' tree', ' and', ' picked']
Tokenized answer: [' an']


Performance on answer token:
Rank: 0        Logit: 20.52 Prob: 64.92% Token: | an|

Top 0th token. Logit: 20.52 Prob: 64.92% Token: | an|
Top 1th token. Logit: 19.53 Prob: 24.22% Token: | a|
Top 2th token. Logit: 17.37 Prob:  2.78% Token: | apples|
Top 3th token. Logit: 17.23 Prob:  2.43% Token: | two|
Top 4th token. Logit: 17.07 Prob:  2.07% Token: | another|
Top 5th token. Logit: 16.00 Prob:  0.71% Token: | some|
Top 6th token. Logit: 15.44 Prob:  0.40% Token: | three|
Top 7th token. Logit: 15.17 Prob:  0.31% Token: | up|
Top 8th token. Logit: 14.97 Prob:  0.25% Token: | the|
Top 9th token. Logit: 14.91 Prob:  0.24% Token: | one|


Ranks of the answer tokens: [(' an', 0)]

In [8]:
prompts = [
"Ross Perot, who ran as",
"Van der Bellen, who ran as",
"Senator Angus King, who ran as",
"Spot gold was up 0.7 percent at $1,340.90",
"An eye for",
"Every action has",
"I'll be there in under half",
"I climbed up the pear tree and picked a pear. I climbed up the apple tree and picked",
"I climbed up the apple tree and picked an apple. I climbed up the pear tree and picked",
]

prompts = [
    "I climbed up the pear tree and picked a pear. I climbed up the apple tree and picked",
    "I climbed up the pear tree and picked a pear. I climbed up the orange tree and picked",
    "I climbed up the apple tree and picked an apple. I climbed up the pear tree and picked",
    "I climbed up the apple tree and picked an apple. I climbed up the banana tree and picked",
    "I climbed up the apple tree and picked an apple. I climbed up the cherry tree and picked",
]

tokens = model.to_tokens(prompts)
final_index = (tokens!=model.tokenizer.pad_token_id).sum(-1)
tokens[np.arange(len(prompts)), final_index]
# tokens[np.arange(len(prompts)), final_index+1]

tensor([6497, 6497, 6497, 6497, 6497], device='cuda:0')

In [9]:
for prompt in prompts:
    utils.test_prompt(prompt, "an", model)

Tokenized prompt: ['<|endoftext|>', 'I', ' climbed', ' up', ' the', ' pear', ' tree', ' and', ' picked', ' a', ' pear', '.', ' I', ' climbed', ' up', ' the', ' apple', ' tree', ' and', ' picked']
Tokenized answer: [' an']


Performance on answer token:
Rank: 0        Logit: 20.52 Prob: 64.92% Token: | an|

Top 0th token. Logit: 20.52 Prob: 64.92% Token: | an|
Top 1th token. Logit: 19.53 Prob: 24.22% Token: | a|
Top 2th token. Logit: 17.37 Prob:  2.78% Token: | apples|
Top 3th token. Logit: 17.23 Prob:  2.43% Token: | two|
Top 4th token. Logit: 17.07 Prob:  2.07% Token: | another|
Top 5th token. Logit: 16.00 Prob:  0.71% Token: | some|
Top 6th token. Logit: 15.44 Prob:  0.40% Token: | three|
Top 7th token. Logit: 15.17 Prob:  0.31% Token: | up|
Top 8th token. Logit: 14.97 Prob:  0.25% Token: | the|
Top 9th token. Logit: 14.91 Prob:  0.24% Token: | one|


Ranks of the answer tokens: [(' an', 0)]

Tokenized prompt: ['<|endoftext|>', 'I', ' climbed', ' up', ' the', ' pear', ' tree', ' and', ' picked', ' a', ' pear', '.', ' I', ' climbed', ' up', ' the', ' orange', ' tree', ' and', ' picked']
Tokenized answer: [' an']


...

Ranks of the answer tokens: [(' an', 0)]

Tokenized prompt: ['<|endoftext|>', 'I', ' climbed', ' up', ' the', ' apple', ' tree', ' and', ' picked', ' an', ' apple', '.', ' I', ' climbed', ' up', ' the', ' cherry', ' tree', ' and', ' picked']
Tokenized answer: [' an']


Performance on answer token:
Rank: 1        Logit: 19.46 Prob: 29.83% Token: | an|

Top 0th token. Logit: 20.02 Prob: 52.23% Token: | a|
Top 1th token. Logit: 19.46 Prob: 29.83% Token: | an|
Top 2th token. Logit: 18.08 Prob:  7.54% Token: | another|
Top 3th token. Logit: 16.89 Prob:  2.29% Token: | two|
Top 4th token. Logit: 16.87 Prob:  2.24% Token: | some|
Top 5th token. Logit: 15.62 Prob:  0.64% Token: | cher|
Top 6th token. Logit: 15.56 Prob:  0.61% Token: | three|
Top 7th token. Logit: 15.32 Prob:  0.47% Token: | cherry|
Top 8th token. Logit: 15.01 Prob:  0.35% Token: | apples|
Top 9th token. Logit: 14.98 Prob:  0.34% Token: | the|


Ranks of the answer tokens: [(' an', 1)]

In [10]:
unembed_dir = model.W_U[:, AN] - model.W_U[:, A]
logits, cache = model.run_with_cache(model.to_tokens(prompts))
resid_stack, resid_labels = cache.get_full_resid_decomposition(expand_neurons=False, apply_ln=True, return_labels=True)
final_resid_stack = resid_stack[:, np.arange(len(prompts)), final_index, :]
(final_resid_stack @ unembed_dir).shape

Tried to stack head results when they weren't cached. Computing head results now


torch.Size([759, 5])

In [12]:
line((final_resid_stack @ unembed_dir).T, line_labels=[p[:10] for p in prompts], x=resid_labels, title="Direct Logit Diff Attribution")


### Look at loss increase from orthogonal component of neuron

In [18]:
d_mlp = model.cfg.d_mlp
final_ln_scale = cache["scale"][np.arange(len(prompts)), final_index, 0]
print(final_ln_scale.shape)
layer = 31

neuron_acts = cache["post", layer][np.arange(len(prompts)), final_index, :]
neuron_wdla = (model.blocks[layer].mlp.W_out @ unembed_dir) / final_ln_scale[:, None]
line(neuron_acts * neuron_wdla, line_labels=[p[:10] for p in prompts], title=f"Neuron DLA in L{layer}")

ni = 892
win = model.blocks[layer].mlp.W_in[:, ni]
wout = model.blocks[layer].mlp.W_out[ni, :]
resid_stack, resid_labels = cache.get_full_resid_decomposition(layer=layer, mlp_input=True, expand_neurons=False, apply_ln=True, return_labels=True)
neuron_resid_stack = resid_stack[:, np.arange(len(prompts)), final_index, :]
print(neuron_resid_stack.shape)

torch.Size([5])


torch.Size([674, 5, 1280])
